In [4]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')
sp500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
sp500


,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


In [11]:
#replace and clean data
sp500['Symbol'] =sp500["Symbol"].str.replace(".",'-')
symbol_list=sp500["Symbol"].unique().tolist()#Grab The listof stock in S&P500
end_date="2024-05-20"
start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

#Download sp500 data from start date to enddate
df=  yf.download(tickers=symbol_list,start=start_date,end=end_date)
df



[*********************100%%**********************]  503 of 503 completed


Price        Adj Close                                                 \
Ticker               A        AAL        AAPL        ABBV        ABNB   
Date                                                                    
2016-05-23   41.823681  30.493269   22.142414   42.024986         NaN   
2016-05-24   42.255348  30.896904   22.479958   43.017139         NaN   
2016-05-25   42.621323  30.916122   22.874907   43.385658         NaN   
2016-05-26   42.705784  30.281841   23.056307   43.867569         NaN   
2016-05-27   42.996670  30.416389   23.042528   44.441597         NaN   
...                ...        ...         ...         ...         ...   
2024-05-13  147.759995  14.930000  186.279999  161.279999  149.130005   
2024-05-14  151.279999  15.190000  187.429993  161.589996  146.699997   
2024-05-15  154.080002  14.940000  189.720001  163.789993  145.800003   
2024-05-16  154.309998  14.790000  189.839996  164.350006  147.190002   
2024-05-17  154.229996  14.730000  189.869995  166.419998  145.660004   

Price                                                                   ...  \
Ticker             ABT        ACGL         ACN        ADBE         ADI  ...   
Date                                                                    ...   
2016-05-23   32.254204   23.576668  102.450447   96.620003   48.317890  ...   
2016-05-24   32.780708   23.936666  105.042999   99.629997   49.560749  ...   
2016-05-25   33.384865   23.900000  104.540367   99.610001   49.963757  ...   
2016-05-26   33.566124   23.956667  104.416931   99.370003   49.980892  ...   
2016-05-27   33.919998   24.053333  104.884285  100.139999   50.109509  ...   
...                ...         ...         ...         ...         ...  ...   
2024-05-13  104.769997   97.910004  307.410004  483.109985  208.380005  ...   
2024-05-14  104.010002   97.900002  306.950012  475.950012  211.940002  ...   
2024-05-15  104.589996   97.870003  308.519989  485.350006  215.750000  ...   
2024-05-16  104.870003   98.820000  308.000000  482.880005  214.119995  ...   
2024-05-17  104.089996  100.989998  303.589996  483.429993  214.080002  ...   

Price       Volume                                                         \
Ticker         WTW       WY     WYNN      XEL       XOM      XYL      YUM   
Date                                                                        
2016-05-23  640800  3477200  2112000  2247600  11766100   713400  4865023   
2016-05-24  761100  4196600  1619800  2358200  10056600  1351300  5034585   
2016-05-25  462100  3404100  3116800  2779500   9578400  1340400  6502925   
2016-05-26  627000  3306300  1974700  2465600   7867900   762800  3582799   
2016-05-27  722500  2887500  1598000  2136400   7505900  1141500  2552207   
...            ...      ...      ...      ...       ...      ...      ...   
2024-05-13  301100  2868600  1750000  4508300  15060600  1276900  1742300   
2024-05-14  570100  3007500   973400  2693900  15079100   949800  2180300   
2024-05-15  355700  4726200  1131500  2278400  18244300  1390200  1810300   
2024-05-16  313700  4181800  1171200  2483500  15745200  1603200  2419700   
2024-05-17  293000  4469400  1071800  4092800  15103100  2266400  1346400   

Price                                 
Ticker          ZBH    ZBRA      ZTS  
Date                                  
2016-05-23  1184912  833600  2356000  
2016-05-24  1665407  589500  3450200  
2016-05-25  1640790  563000  4637700  
2016-05-26  1341163  452200  3116900  
2016-05-27  1053381  611300  2023200  
...             ...     ...      ...  
2024-05-13  1180900  232900  2125800  
2024-05-14  1348700  216300  2050100  
2024-05-15  1153900  217500  2585700  
2024-05-16   970800  288400  1694900  
2024-05-17  1181200  222700  1999500  

[2011 rows x 3018 columns]

In [19]:

df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-05-23 A        41.823681   44.570000   44.959999   44.529999   44.889999   
           AAL      30.493269   31.730000   32.290001   31.700001   32.029999   
           AAPL     22.142414   24.107500   24.297501   23.917500   23.967501   
           ABBV     42.024986   59.299999   59.840000   59.130001   59.750000   
           ABT      32.254204   37.369999   37.750000   37.270000   37.720001   
...                       ...         ...         ...         ...         ...   
2024-05-17 XYL     143.320007  143.320007  143.380005  141.960007  143.210007   
           YUM     141.660004  141.660004  141.710007  140.220001  141.500000   
           ZBH     120.180000  120.180000  121.300003  119.559998  121.300003   
           ZBRA    317.890015  317.890015  320.739990  317.519989  320.739990   
           ZTS     174.059998  174.059998  174.660004  172.580002  174.050003   

Price                   volume  
date       ticker               
2016-05-23 A         1689700.0  
           AAL       6498400.0  
           AAPL    152074400.0  
           ABBV      6468900.0  
           ABT       7126700.0  
...                        ...  
2024-05-17 XYL       2266400.0  
           YUM       1346400.0  
           ZBH       1181200.0  
           ZBRA       222700.0  
           ZTS       1999500.0  

[991899 rows x 6 columns]

Calculate featuresand technical indicators for each stock

*GARMAN KLASS VOLATILITY
Garman Klass is a volatility estimator that incorporates open, low, high, and close prices of a security.

*RSI
Overbought and Oversold Levels:

Overbought: An RSI value over 70 suggests that the stock or asset is becoming overvalued and may be primed for a trend reversal or corrective pullback in price. It's a sell signal for some traders.
Oversold: An RSI value below 30 indicates an undervalued condition, suggesting a potential turnaround point where buyers might step in, seeing it as an opportunity to buy.
Centerline Crossover:

RSI values greater than 50 suggest a bullish market momentum, while values below 50 indicate bearish momentum.


In [22]:
#GARMAN _KLASS_VOLITILITY
df['garman_klass_vol'] = ((np.log(df['high']) - np.log(df['low']))**2) / 2 - (2 * np.log(2) - 1) * (np.log(df['adj close']) - np.log(df['open']))**2
df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-05-23 A        41.823681   44.570000   44.959999   44.529999   44.889999   
           AAL      30.493269   31.730000   32.290001   31.700001   32.029999   
           AAPL     22.142414   24.107500   24.297501   23.917500   23.967501   
           ABBV     42.024986   59.299999   59.840000   59.130001   59.750000   
           ABT      32.254204   37.369999   37.750000   37.270000   37.720001   
...                       ...         ...         ...         ...         ...   
2024-05-17 XYL     143.320007  143.320007  143.380005  141.960007  143.210007   
           YUM     141.660004  141.660004  141.710007  140.220001  141.500000   
           ZBH     120.180000  120.180000  121.300003  119.559998  121.300003   
           ZBRA    317.890015  317.890015  320.739990  317.519989  320.739990   
           ZTS     174.059998  174.059998  174.660004  172.580002  174.050003   

Price                   volume  garman_klass_vol  
date       ticker                                 
2016-05-23 A         1689700.0         -0.001888  
           AAL       6498400.0         -0.000764  
           AAPL    152074400.0         -0.002299  
           ABBV      6468900.0         -0.047766  
           ABT       7126700.0         -0.009384  
...                        ...               ...  
2024-05-17 XYL       2266400.0          0.000049  
           YUM       1346400.0          0.000055  
           ZBH       1181200.0          0.000071  
           ZBRA       222700.0          0.000020  
           ZTS       1999500.0          0.000072  

[991899 rows x 7 columns]

In [ ]:
#RSI

